In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import csv
headers = {
  'User-Agent': 'Mozilla/5.0',
  'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}


year = []
prices = []
name =[]
types = []
seatQuantity =[]
motors = []
material = []
capacity = []
origin =[]
gears =[]
company =[]



def formatEngine(string):
    words = string.split()
    if(len(words) == 1):
        material = string
        capacity = 0
    else:
        material = words[0]
        capacity = words[1]
    return material, capacity


def formatCompany(data):
  return data.split(' ')[0]

def formatCarName(data):
  return data.replace(formatCompany(data),'')


def formatCurrency(number):
    number = number.replace(" ", "")
    if "Tỷ" in number and "Triệu" in number:
        billion = int(number.split("Tỷ")[0])
        million = int(number.split("Tỷ")[1].replace("Triệu", ""))
        value = billion * 1000000000 + million * 1000000
    elif "Tỷ" in number:
        billion = int(number.replace("Tỷ", ""))
        value = billion * 1000000000
    else:
        million = int(number.replace("Triệu", ""))
        value = million * 1000000
    return '{:,}'.format(value)

def crawl(page):

  req = requests.get('https://bonbanh.com/oto-xe-moi-gia-duoi-1600-trieu/page,{}'.format(page), headers=headers)  # URL of the website which you want to scrape
  content = req.content
  soup = BeautifulSoup(content, 'html.parser')
  products = soup.find_all('li', class_='car-item')
  for product in products:
      p_title =  product.find('h3').text.strip()
      p_year = product.find('div', class_='cb1').find('b').text.strip()
      p_price = product.find('div', class_='cb3').find('b').text.strip()
      year.append(p_year)
      prices.append(formatCurrency(p_price))
      name.append(formatCarName(p_title))
      company.append(formatCompany(p_title))
      p_url = product.find('a').get('href')
      detaiLCar = requests.get('https://bonbanh.com/{}'.format(str(p_url)),headers=headers)
      detailSoup = BeautifulSoup(detaiLCar.content, 'html.parser')
      details = detailSoup.find_all('div',class_='row')
      p_seat = detailSoup.find('div', class_='row_last').find('span', class_='inp').text.strip()
      seatQuantity.append(p_seat.replace('chỗ',''))
      for detail in details:
        p_label = detail.find('label').text.strip()
        p_data = detail.find('span', class_='inp').text.strip()
        if('Dòng xe' in p_label):
          types.append(p_data)
        if('Động cơ' in p_label):
          [t_material, t_capacity] = formatEngine(p_data)
          material.append(t_material)
          capacity.append(t_capacity)
        if('Xuất xứ' in p_label):
          origin.append(p_data)
        if('Hộp số' in p_label):
          gears.append(p_data)
          


      

def save(name,  prices, year,types, seatQuantity, material, capacity,origin, gears,company):
  data = {
      'Hãng xe':company,
      'Tên xe': name,
      'Giá (VNĐ)': prices,
      'Năm sản xuất': year,
      'Dòng xe': types,
      'Số chỗ ngồi': seatQuantity,
      'Nguyên liệu': material,
      'Dung tích (L)': capacity,
      'Xuất xứ': origin,
      'Hộp số': gears,
  }
  df = pd.DataFrame.from_dict(data, orient='index')

  df = df.transpose()
  dataset = pd.DataFrame(data=df)
  dataset.to_csv('car-data.csv')
  dataset.to_excel('data.xlsx', index=False)
  

  csv_reader = csv.reader('car-data.csv',delimiter=",")


if __name__ == '__main__':
  print(name)
  for i in range(1, 300):
    crawl(i)
  crawl(1)
  save(name, prices,year,types,seatQuantity, material, capacity,origin, gears,company)


[]


In [ ]:

def formatEngine(string):
    words = string.split()
    if(len(words) == 1):
        material = string
        capacity = 0
    else:
        material = words[0]
        capacity = words[1]
    return material, capacity

print(formatEngine('Hybird'))

('Hybird', 0)
